# *How Removing A 45-Year-Old Rule Would Bring Down Concussion Rates*

By Alex Wainger (Data Engineer, Facebook)

## Proposed Rule Change
Eliminate the rule that kicking team players cannot go more than a yard past the line of scrimmage before a punt.$^{[1]}$

## How Does That Help?
Removing this rule will increase the fair-catch rate and discourage teams from returning punts, which is the most dangerous part of the play. Allowing the kicking team to release downfield before the ball leaves the punter's foot **will reduce punt-related concussions by ~33%** while minimally impacting the receiving team's starting field position and the occurrence of exciting returns.

## A Brief History Of The Existing Rule
The NFL introduced this rule before the 1974 season,$^{[2]}$ inspired by a similar rule change made by the NCAA in 1967.$^{[3]}$ The rule was part of "a package of changes to reinvigorate the game by...increasing the opportunity for big plays on kickoff and punt returns..."$^{[4]}$ in an effort to combat the launch of the World Football League — a rival league that "sought to attract fans with rules that were friendlier to the offense than the NFL’s" — in the year prior.$^{[4]}$ 

However, even in 1974, some were concerned about the health implications of this new rule. Bill Curry, President of the Players Association at the time, said "the possibility of wide‐open punt returns with bodies crashing downfield is anathema to the players."$^{[5]}$ Despite the concerns, the rule change was approved and has now been part of the NFL rulebook for 45 years.

Sources:
1. https://operations.nfl.com/the-rules/2018-nfl-rulebook/#article-2.-kicking-team-players-on-line-during-kick
2. https://en.wikipedia.org/wiki/1974_NFL_season#Major_rule_changes
3. https://en.wikipedia.org/wiki/1967_college_football_season#Rule_changes
4. https://operations.nfl.com/the-rules/evolution-of-the-nfl-rules
5. https://www.nytimes.com/1974/05/21/archives/changes-bynfl-opposed-nfl-rule-changes-bring-opposition.html

In [ ]:
import feather
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy.stats import linregress

from scipy.stats import ttest_ind
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV

%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

## Exploratory Data Analysis
I'll spare you from going through all the exploration and visualization I did for this project. Instead, I'll show a couple relevant charts and numbers that inspired my proposed rule change.

In [ ]:
# Loading all the datasets I built in the preprocessing notebook
play_information = feather.read_dataframe('../input/nflpuntpreprocessed/play_information.feather')
game_data = feather.read_dataframe('../input/nflpuntpreprocessed//game_data.feather')
play_player_role_data = feather.read_dataframe('../input/nflpuntpreprocessed/play_player_role_data.feather')
min_distances = feather.read_dataframe('../input/nflpuntpreprocessed/min_distances.feather')
second_min_distances = feather.read_dataframe('../input/nflpuntpreprocessed/second_min_distances.feather')
punt_hangtime = feather.read_dataframe('../input/nflpuntpreprocessed/punt_hangtime.feather')
video_review = feather.read_dataframe('../input/nflpuntpreprocessed/video_review.feather')

In [ ]:
# Building a master play-level dataset
aug_play_information = play_information.merge(
    min_distances[['GameKey', 'PlayID', 'Coverage_Distance']],
    on=['GameKey', 'PlayID'], how='left', validate='one_to_one'
).merge(
    second_min_distances[['GameKey', 'PlayID', 'Coverage_Distance']],
    on=['GameKey', 'PlayID'], how='left', validate='one_to_one',
    suffixes=['_1', '_2']
).merge(
    game_data[['GameKey', 'Temperature', 'Is_Grass', 'Is_Outdoor']] ,
    on=['GameKey'], how='left', validate='many_to_one'
).merge(
    punt_hangtime[['GameKey', 'PlayID', 'Hangtime']],
    on=['GameKey', 'PlayID'], how='left', validate='one_to_one'
).merge(
    video_review,
    on=['GameKey', 'PlayID'], how='left', validate='one_to_one',
)

aug_play_information['Has_Concussion'] = ~aug_play_information.GSISID.isna()

In [ ]:
plays_with_punts = aug_play_information[aug_play_information.Has_Punt]
plays_with_concussions = aug_play_information[aug_play_information.Has_Concussion]

print("Total plays:", aug_play_information.shape[0])
print("Plays with a punt:", plays_with_punts.shape[0])
print("Plays with a concussion:", plays_with_concussions.shape[0])
print("Plays with a punt and a concussion:", plays_with_concussions[plays_with_concussions.Has_Punt].shape[0])

The 219 plays without a punt are mostly fakes and plays blown dead by penalties, and with only 1 concussion, I'm going to exclude them from the remainder of the analysis and focus on plays in which a punt actually occurs.

I looked at a bunch of slices of these 36 punt-related concussions, and one stood out in particular — concussion rates broken down by how the receiving team fielded the punt. I categorized each punt return type into 1 of 3 groups — fair catch, return, or unreturnable (i.e. punts out of bounds, downed punts, touchbacks, etc).

In [ ]:
plays_with_concussions = plays_with_concussions[plays_with_concussions.Has_Punt]

concussion_rates = plays_with_concussions.Punt_Type.value_counts() / plays_with_punts.Punt_Type.value_counts()
ax = concussion_rates.plot(
    kind='barh', figsize=(10,5), color='#2678B2', fontsize=12,
    title='Concussions Occur Over 7x More Often On Returned Punts')
vals = ax.get_xticks()
ax.set(xlabel="Concussion Rate", xticklabels=['{:,.1%}'.format(x) for x in vals]);
ax.xaxis.label.set_size(14)
ax.title.set_size(16)

ttest_res = ttest_ind(
    plays_with_punts[plays_with_punts.Has_Return].Has_Concussion, 
    plays_with_punts[~plays_with_punts.Has_Return].Has_Concussion
)

if ttest_res.pvalue < .05:
    print('The difference in concussion rates on plays with and without returns is statistically significant.')
else:
    print('The difference in concussion rates on plays with and without returns is not statistically significant.')
print('p-value:', '%f' % ttest_res.pvalue)

With a concussion rate of over 1%, a player is over 7x more likely to get concussed on a punt play with a return than a play without one (and that difference is statistically significant).

To validate this finding, I reviewed all of the videos of the plays with concussions. Of the 31 concussions that occurred on plays with returns, **30 of the concussions occurred after the punt was caught by the returner.** This point is essential to the rest analysis — fair catch plays and return plays are essentially identical up until the moment the punt returner catches the ball. In one, the play ends the moment the ball is caught, in the other, players run full-speed into one another from all sides.

Therefore, incentivizing punt returners to call more fair catches, rather than returning punts, should reduce the overall concussion rate, as the return play concussions would not happen if the play ended when the punt was caught.

## Fair Catch Model
In order to incentivize teams to call more fair catches, I first had to understand what the drivers of fair catches were.

I had a hypothesis that the distance between the punt returner and the nearest member of the kicking team would give us a lot of information about whether or not a fair catch was called. To prove it, I built a model to predict whether or not a punt play ended with a fair catch.

### Features

 - **Coverage_Distance_1 and Coverage_Distance_2:** Coverage distance is the distance between the punt returner and a member of the coverage team at the moment the punt is caught. Each play has 11 coverage distance values (one for each member of the coverage team), so we pick the smallest and second smallest and assign them the \_1 and \_2 subscripts, respectively. 
 - **Punt_Distance:** How many yards the punt traveled.
 - **Time_Passed_Sec:** How many seconds of game-time have passed when the punt is happening. 0 and 3600 represent the start and end of the game, 900, 1800, and 2700 represent the start of the second, third, and fourth quarters, etc.
 - **Score_Differential:** kicking team score - receiving team score
 - **Yard_Line_Absolute:** The line of scrimmage, represented as a number between 0 and 100. 0 represents the kicking team's goal line, 100 represents the receiving team's goal line.
 - **Hangtime:** Amount of time between ball leaving punter's foot and the punter returning catching it.
 - **Temperature:** For outdoor games, it's the temperature outside, for indoor games, 70 was used to represent an average indoor stadium temperature (as we don't have the inside temperature available)
 - **Is_Grass:** Boolean, whether or not the game is being played on grass.
 - **Is_Outdoor:**: Boolean, whether or not the game is being played outdoors.

In [ ]:
plays_with_return = plays_with_punts[plays_with_punts.Has_Return]
plays_with_fair_catch = plays_with_punts[plays_with_punts.Has_Fair_Catch]

label = ['Has_Fair_Catch']

features = [
    'Punt_Distance', 'Time_Passed_Sec', 'Score_Differential',
    'Yard_Line_Absolute', 'Coverage_Distance_1', 'Coverage_Distance_2',
    'Hangtime', 'Temperature', 'Is_Grass', 'Is_Outdoor'
]

plays_with_return_or_fc = pd.concat(
    [plays_with_return, plays_with_fair_catch]
)[['GameKey', 'PlayID'] + features + label].dropna()

x_train, x_test, y_train, y_test = train_test_split(
    np.array(plays_with_return_or_fc[features]),
    np.array(plays_with_return_or_fc[label]).ravel(),
    test_size=0.2,
    random_state=0
)

In [ ]:
parameters = {
    'n_estimators':[10, 50, 100],
    'learning_rate': [.01, .05, .1],
    'max_depth': [1, 2, 3]
}

search = GridSearchCV(GradientBoostingClassifier(random_state=0), parameters, cv=3)
search.fit(x_train, y_train)

print(search.best_params_)

The model is trained on punts with returns as negative examples (unreturnable punts aren't really relevant here, since we're trying to learn how to convert returns to fair catches) and performs pretty well! Given the sample sizes we're working with, the data we have available and it's reliability, and the amount of actual randomness in the decision to call a fair catch, I'm happy with the accuracy.

In [ ]:
# Using best params from grid-search above
gbdt = GradientBoostingClassifier(
    n_estimators=search.best_params_['n_estimators'],
    learning_rate=search.best_params_['learning_rate'],
    max_depth=search.best_params_['max_depth'],
    random_state=0
).fit(x_train, y_train)

y_scores_gbdt = gbdt.predict_proba(x_test)[:,1]

print("Train accuracy: {:,.1%}".format(gbdt.score(x_train, y_train)))
print("Test accuracy: {:,.1%}".format(gbdt.score(x_test, y_test)))
print("ROC AUC: {:,.3f}".format(roc_auc_score(y_test, y_scores_gbdt)))

Looking at the feature importances for our fair-catch model, my original hypothesis was correct — the minimal coverage distance is by far the most important feature in predicting whether or not a fair catch was called.

In [ ]:
ax = pd.DataFrame(
    gbdt.feature_importances_,
    index = features,
    columns=['importance']
).sort_values('importance', ascending=True).plot(
    title='Fair-Catch Model Feature Importances', kind='barh', legend=False, figsize=(10,5), fontsize=12
)

ax.title.set_size(16)

So we know coverage distance is by far the most important feature in predicting fair catches. Logically, there is probably an inverse relationship between coverage distance and fair catches, i.e. smaller coverage distance —> larger odds of a fair catch. To confirm this, I trained a logistic regression model to perform the same task and looked at the direction of the coefficients -- unsurprisingly, coverage distance has a negative coefficient.

In [ ]:
logreg = LogisticRegression(solver='liblinear', random_state=2).fit(x_train, y_train)
y_scores_logreg = logreg.predict_proba(x_test)[:,1]
y_pred_logreg = logreg.predict(x_test)

print("Train accuracy:",logreg.score(x_train, y_train))
print("Test accuracy:", logreg.score(x_test, y_test))
print("ROC AUC:", roc_auc_score(y_test, y_scores_logreg))

ax = pd.DataFrame(
    logreg.coef_.transpose(),
    index=features,
    columns=['coefficients']
).sort_values('coefficients').plot(
    kind='barh', legend=False, title='Logistic Regression Feature Coefficients', figsize=(10,5), fontsize=14
)

ax.title.set_size(16)


## Estimating Impact of the Rule Change
Knowing what we now know about fair catches, the rule holding non-gunners at the line of scrimmage until the punt stood out as an area of opportunity — it gives returners space to advance the ball, which we know is associated with higher concussion rates.

Removing the rule would allow players to release downfield into coverage whenever they want. It will strictly decrease the minimal coverage distance — conservative teams afraid of punt blocks will hold their blocking as long as they do now, while more aggressive teams will leave the line of scrimmage well before the punt leaves the punter's foot.

To estimate how many returns get converted into fair catches, I did the following:
1. Measure the amount of time between the snap and punt events.
2. Calculate the average speed of each non-gunner on the kicking team between the punt and the reception.
3. Picking some fraction to represent how much sooner the kicking team would start downfield, calculate "adjusted coverage distances" —
   1. For gunners, this is the same as the coverage distance we originally calculated for the model
   2. For non-gunners, *adjusted_coverage_distance = coverage_distance - (snap_to_punt_time X average_speed X blocking_to_coverage_time_fraction)*
4. Feed the smallest and second smallest adjusted coverage distances back into our trained fair catch model that we developed earlier and see how many returns are now predicted to be fair catches.

I picked 0.5 for the blocking_to_coverage_time_fraction, which means in my simulation, the non-gunners will start heading downfield half way between the snap and the punt. As I wrote above, conservative teams may not send their players much earlier, while aggressive teams may just chip the receiving blockers and then head downfield, so 0.5 seemed like a reasonable middle ground.

In [ ]:
def update_coverage_distance(row):
    snap_to_punt_time_adjustment = .5 # Percentage of blocking time converted to coverage time
    minimal_coverage_distance = .5 # don't let adjusted coverage distance get smaller than half a yard
    if 'GL' in row.Role_k or 'GR' in row.Role_k:
        return row.Coverage_Distance
    else:
        return max(
            row.Coverage_Distance - (
                row.Yards_Per_Second * (
                    row.Snap_To_Punt_time * snap_to_punt_time_adjustment
                )
            ), minimal_coverage_distance
        )

In [ ]:
# Calculate Adjusted Coverage Distances
adjusted_coverage_distances = feather.read_dataframe('../input/nflpuntpreprocessed/adjusted_coverage_distances.feather')

adjusted_coverage_distances['Adj_Coverage_Distance'] = adjusted_coverage_distances.apply(
    update_coverage_distance, axis=1
)

min_acd = adjusted_coverage_distances.loc[
    adjusted_coverage_distances.groupby(['GameKey', 'PlayID'])['Adj_Coverage_Distance'].idxmin()
][['GameKey', 'PlayID', 'Adj_Coverage_Distance', 'Role_k', 'Super_Role_k']]

adjusted_coverage_distances_2 = adjusted_coverage_distances.drop(
    adjusted_coverage_distances.groupby(['GameKey', 'PlayID'])['Adj_Coverage_Distance'].idxmin()
)

second_min_acd = adjusted_coverage_distances_2.loc[
    adjusted_coverage_distances_2.groupby(['GameKey', 'PlayID'])['Adj_Coverage_Distance'].idxmin()
][['GameKey', 'PlayID', 'Adj_Coverage_Distance', 'Role_k', 'Super_Role_k']]

Looking first at the breakdown of which member of the coverage team is closest to the punt returner at punt reception time, we can see that the rule change drastically changes the distribution. In the game today, gunners are the closest player nearly 80% of the time, but that number gets nearly cut in half after my rule change. This is a good sign for the rule change, as it means the minimum coverage distance is getting smaller, as linemen and protectors are getting to the punt returner before the gunners much more often than before the rule change.

In [ ]:
ax = pd.DataFrame({
    "Before Rule Change": min_distances.Super_Role_k.value_counts() / min_acd.shape[0],
    "After Rule Change": min_acd.Super_Role_k.value_counts() / min_acd.shape[0]
}).plot(kind='barh', figsize=(10,5), title='Nearest Coverage Team Member At Time Of Punt Reception')

vals = ax.get_xticks()
ax.set(xlabel="% of Plays", xticklabels=['{:,.0%}'.format(x) for x in vals])

ax.title.set_size(16)
ax.xaxis.label.set_size(14)

In [ ]:
# Evaluate trained model on new adjusted coverage distance features
adj_features = [
    'Punt_Distance', 'Time_Passed_Sec', 'Score_Differential',
    'Yard_Line_Absolute', 'Adj_Coverage_Distance_1', 'Adj_Coverage_Distance_2',
    'Hangtime', 'Temperature', 'Is_Grass', 'Is_Outdoor'
]

adj_X = plays_with_return_or_fc.merge(
    min_acd, on=['GameKey', 'PlayID'], how='inner', validate='one_to_one'
).merge(
    second_min_acd, on=['GameKey', 'PlayID'], how='inner', validate='one_to_one', suffixes=["_1", "_2"]
)

adj_X['Adj_Has_Fair_Catch_pred'] = gbdt.predict(adj_X[adj_features]) | adj_X.Has_Fair_Catch

In [ ]:
adj_plays_with_punts = plays_with_punts.merge(
    adj_X[['GameKey', 'PlayID', 'Adj_Has_Fair_Catch_pred']],
    on=['GameKey', 'PlayID'], how='left', validate='one_to_one'
)

adj_plays_with_punts['Adj_Punt_Type'] = adj_plays_with_punts.apply(
    lambda row:
        row.Punt_Type
        if pd.isna(row.Adj_Has_Fair_Catch_pred)
        else (
            'fair catch'
            if row.Adj_Has_Fair_Catch_pred
            else 'return'
        )
    , axis=1
)
adj_plays_with_punts['Adj_Has_Fair_Catch_pred'] = adj_plays_with_punts.Adj_Punt_Type == 'fair catch'
adj_plays_with_punts['Adj_Punt_Return_Length'] = adj_plays_with_punts.apply(
    lambda row:
        row.Punt_Return_Length
        if not row.Adj_Has_Fair_Catch_pred
        else 0.0
    , axis=1
)

In [ ]:
ax = pd.DataFrame({
    "(Estimated) After Rule Change": adj_plays_with_punts.Adj_Punt_Type.value_counts() / adj_plays_with_punts.shape[0],
    "Before Rule Change": adj_plays_with_punts.Punt_Type.value_counts() / adj_plays_with_punts.shape[0]
}, index=adj_plays_with_punts.Punt_Type.unique()).transpose().plot(
    kind='barh', title='Rule Change Increases Fair Catch Frequency from 25% to 40%', figsize=(10,5)
)
vals = ax.get_xticks()
ax.set(xlabel="% of Plays", xticklabels=['{:,.0%}'.format(x) for x in vals]);

ax.xaxis.label.set_size(14)
ax.title.set_size(16)

By reevaluating the model with the adjusted coverage distance features, we can essentially simulate the two seasons of punting with my rule change and see what kind of effects it has on fair-catch frequency. As expected, the percent of plays that end with a fair catch jumps from 25% to 40% and becomes the most common way to end a punt play.

### Concussion Rate

In [ ]:
pre_concussions = adj_plays_with_punts.Has_Concussion.sum()
post_concussions = (
    adj_plays_with_punts.Has_Concussion & ~(
        (adj_plays_with_punts.Punt_Type == 'return') & (adj_plays_with_punts.Adj_Punt_Type == 'fair catch')
    )
).sum()
print("# Concussions Before Rule Change:", pre_concussions)
print("# Concussions After Rule Change:", post_concussions)
print("Percent Decrease in Concussions: {:,.1%}".format((pre_concussions - post_concussions) / pre_concussions))

To understand the impact that this rule change could have on the occurrences of concussions, I identified all of the concussions that happened on returns that my rule change would have converted to fair catches — these would not have happened in this alternate universe. This is obviously a rough estimate, as there are a number of assumptions and randomness baked into the analysis, but the result shows that converting returns to fair catches could have a significant impact on overall concussion rates.

### Field Position and "Exciting" Returns
So concussion rates are going down, great! But what if this rule change takes away all the excitement of punt returns, or drastically changes the average starting field position for the receiving team? Luckily, the increases in fair catches doesn't have too big an impact on either.

In [ ]:
adj_plays_with_punts['Next_Poss_Yard_Line'] = adj_plays_with_punts.apply(
    lambda row:
        np.nan
        if row.Punt_Type == 'unreturnable'
        else 100 - (row.Yard_Line_Absolute + row.Punt_Distance - row.Punt_Return_Length)
    , axis=1
)

adj_plays_with_punts['Adj_Next_Poss_Yard_Line'] = adj_plays_with_punts.apply(
   lambda row:
        np.nan
        if row.Adj_Punt_Type == 'unreturnable'
        else 100 - (row.Yard_Line_Absolute + row.Punt_Distance - row.Adj_Punt_Return_Length)
    , axis=1
)

adj_plays_with_returnable_punts = adj_plays_with_punts[
    adj_plays_with_punts.Has_Return | adj_plays_with_punts.Has_Fair_Catch
]

ax = pd.DataFrame(
    {
        'Before Distribution':adj_plays_with_returnable_punts.Next_Poss_Yard_Line,
        'After Distribution': adj_plays_with_returnable_punts.Adj_Next_Poss_Yard_Line
    }
).plot(
    kind='density', xlim=(0,100), ylim=(0,.03), figsize=(12,6),
    title='Distribution of Receiving Team\'s Starting Field Position',
)

before_mean_yard_line = adj_plays_with_returnable_punts.Next_Poss_Yard_Line.mean()
after_mean_yard_line = adj_plays_with_returnable_punts.Adj_Next_Poss_Yard_Line.mean()

yard_line_vals = [0,10,20,30,40,50,60,70,80,90,100]
yard_line_labels = [
    'Own Goal Line' if x == 0
    else (
        'Opp Goal Line' if x == 100
        else (
            x if x <= 50 
            else 100-x
        )
    ) for x in yard_line_vals]
ax.set(
    xlabel="Yard Line", xticks=yard_line_vals, xticklabels=yard_line_labels,
    yticklabels= [''] + [str(x) for x in ax.get_yticks()[1:]]
)
plt.axvline(
    before_mean_yard_line, linestyle='dashed', color='tab:blue', linewidth=2,
    label='Before Mean: {:,.1f} yard line'.format(before_mean_yard_line)
)
plt.axvline(
    after_mean_yard_line, linestyle='dashed', color='tab:orange', linewidth=2,
    label='After Mean: {:,.1f} yard line'.format(after_mean_yard_line)
)
ax.xaxis.label.set_size(14)
ax.title.set_size(16)
plt.legend();

Average starting field position drops by just a yard and a half, from the 28.4 to the 27. After the rule change, we see a small spike around the 15 and a small decrease in frequency of drives starting between the 40's. These are pretty minimal effects given the amount we're reducing concussions by.

A savvy reader may notice that I'm assuming punt return distances wouldn't be affected by the rule change. I originally built a model using my coverage distance features to predict punt return distance, thinking that the closer the coverage team was to the returner when he caught the ball, the smaller the return will be. That model performed pretty poorly, and calculating a simple correlation between coverage distance and punt return distance shows why:

In [ ]:
x = plays_with_return[['Punt_Return_Length', 'Coverage_Distance_1']].dropna()
linregress_result = linregress(x.Coverage_Distance_1, x.Punt_Return_Length)
print('R^2: {:,.3f}'.format(linregress_result.rvalue ** 2))
print('P-value: {}'.format(linregress_result.pvalue))

Just 4.2% of the variance in punt return return distance can be explained by the minimal coverage distance. The p-value is extremely small, meaning that the minimum coverage distance would be a meaningful addition to a model predicting punt return distance, but there is a ton of variance in punt return distance that coverage distance just can't explain.

The primary impact of my rule change would be in reducing the minimum coverage distance, and there are so many other factors that influence how far a punt gets returned, so it would have just added noise to the field position analysis to include an adjustment from my rule change.

In [ ]:
exciting_return_length = 20

print("Of All Punts (returnable and unreturnable)")
print("    Percent of total punt plays with >20 yard return before change: {:,.1%}".format(
    (adj_plays_with_punts.Punt_Return_Length > exciting_return_length).sum() / adj_plays_with_punts.shape[0]
))
print("    Percent of total punt plays with >20 yard return after change: {:,.1%}".format(
    (adj_plays_with_punts.Adj_Punt_Return_Length > exciting_return_length).sum() / adj_plays_with_punts.shape[0]
))

Looking now at "exciting" punt returns (defined here as a return of 20 yards or more), we see the percent of punt plays with one drops by a percentage point, from 4.1% to 3.1%. While in the abstract this represents a ~25% decrease in exciting returns, the fact that only 1 in 25 punts end up having an exciting return to begin with likely means fans won't notice the downtick, as long as there is still a big run back every now and then. Tyreek Hill and Odell Beckham will still be able to break off electrifying returns that set their teams up with good field position.

## Unintended Consequences

In terms of effects I can reasonably estimate with data, the rule change seems to do exactly what I want it to do. However, there would almost certainly be second- and third-order effects or unintended consequences of the rule change. It's hard (or even impossible) to quantify how each of these might affect the game, but I don't think that even in a worst-case scenario that any of them outweigh the reduction in concussions.

#### NCAA-style Rugby Roll Out Punts
Since the NCAA allows the kicking team to release downfield on the snap, we can learn a lot from how college teams cover punts. The most obvious difference between the college and professional punting game is the use of a rugby-style punt — some college teams will roll their punter out toward the sideline, buying time for the rest of the coverage team to sprint further downfield than they would during a standard 3-step punt. My rule change could cause an inadvertent rise in this type of strategy.

However, I don't actually think rugby roll out punts would catch on in the NFL. College punters often don't have the leg strength to hit a 50 yard punt with 4-5 seconds of hang time. As a result, teams turn to a gimmicky play to try to buy more time for their coverage team. At the NFL level, punters can boom kicks far and high enough that coverage teams have time to get downfield with just 1-2 seconds from snap to kick (and they'll have even more time under my rule change).

Interestingky, last season Texas and Alabama had the 1st and 3rd ranked punters in the country in Michael Dickson and JK Scott (both of whom are now starters in the NFL), and both teams primarily kicked traditional punts — despite Dickson having played Australian-rules football until he was 19. If your punter has the leg strength to boom the ball down the field, there's no point in trying to pull off a risky on-the-move punt to buy a few extra seconds.

Relatedly, the rugby roll out also requires a very particular skill-set — ability to punt (or at least threaten to punt) with either leg while on the run — that most NFL punters likely don't practice (Australians like Dickson being the exception here, but while they're extremely common in college, there aren't an overwhelming number of starting Australian punters in the NFL today). Initially, teams may try the tactic, but it would take a few seasons of experimenting and practice under the rule change to master the rugby roll out, at which point the NFL would have some data to evaluate whether or not it's good for the game.

#### Changes In Receiving Team Strategy
Another side effect I could see arising from my rule change is the receiving team changing the way they approach blocking for the punt returner. Some teams may choose to drop a few blockers back near the punt returner to give the returner time and space to advance the ball. This could potentially dampen my rule change's reduction in returns (which in turn would reduce the impact on overall concussion rates).

However, if the return team goes overboard with dropping blockers back, they will leave themselves susceptible to fake punts. For example, if the return team lines up with a returner, 2 gunners, and 3 "personal protectors" for the returner who are somewhere between the returner and the line of scrimmage, that would leave just 5 down linemen at the line. Traditional formations for the kicking team have two gunners, a punter, and 8 linemen, wings, and protectors all near the ball on the line of scrimmage. An 8 vs. 5 advantage at the line would make it trivially easy for the kicking team to direct snap to a skill-position player, overpower the 5 down linemen, and pickup a first down. As a result, I wouldn't be worried about teams dropping a significant number of players away from the line.

In the opposite extreme, some teams may abandon the punt return all together and instead send all their players after a punt block, always calling a fair catch if they don't block the punt (since there would be no blocking for the returner). This poses a legitimate threat to my rule change's impact, as kicking teams may be forced to hold their blocks until the punt, which would give the returner the same time and space he has now to attempt a return.

If we saw teams employing this strategy after my rule change gets implemented, it would be pretty easy to pass a formation rule that prevents the receiving team from sending everyone after the punt (e.g. the receiving team has to at least match the number of players the kicking team has outside the numbers, so two gunners must be matched by two jammers) to mitigate this strategy.

## Conclusion

So there you have it! The removal of a 45-year-old rule could help prevent around 1 out of 3 punt-related concussions while minimally impacting the integrity and quality of the return game. I'll leave you with one final advantage of my rule change — it takes something out of the rulebook. I brainstormed a ton of reward-based rules like gifting returners some amount of yards for doing some action, or adding additional penalties to try to take away certain types of hits. Even if those rule changes are just as effective as mine, they also makes the game more complicated. 

If we continue down the path of adding new rules to incentivize players to do what we want, we'll eventually end up with a complicated web of rewards and penalties, with each new rule addition becoming harder to evaluate and implement than the last. A rule change that effectively reduces concussions while also making the game just touch less complicated is a win-win in my book.